## SmolVLM 256M Instruct

In [10]:
import torch
import time
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# Load images
image = Image.open("../tasks/bird.jpg")

In [3]:
# Initialize processor and model
processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-256M-Instruct")

model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-256M-Instruct",
    torch_dtype=torch.bfloat16,
).to(DEVICE)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [6]:
# Create input messages
query = "Can you describe the image?"
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": query}
        ]
    },
]
# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image], return_tensors="pt")
inputs = inputs.to(DEVICE)

In [15]:
# Generate outputs
t1 = time.time()
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)
t2= time.time()
print(generated_texts[0])
print(f"\n Generation Time: {round(t2-t1, 2)}s")

User:



Can you describe the image?
Assistant: The image features a hummingbird in flight. The hummingbird is in the foreground and is flying towards the right side of the image. It has a long, thin beak and is colored in a mix of green and blue. The hummingbird's wings are spread out, and its tail is visible, indicating it is flying. The background of the image is blurred, focusing the viewer's attention on the hummingbird. The background color is a gradient of light green to yellow, which is a common color used in nature to create a sense of depth and movement. The overall composition of the image is balanced, with the hummingbird in the foreground and the background being slightly blurred, creating a sense of depth.

 Generation Time: 15.67s
